In [1]:
import os
import openai
import logging
import time
import requests
from openai import OpenAI, APIConnectionError, RateLimitError, APIError

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# OpenAI API setup
openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    logging.error("OpenAI API key not found. Please set the 'OPENAI_API_KEY' environment variable.")
    exit(1)

client = OpenAI(api_key=openai_api_key)

output_directory = os.path.join('3.5T Blog Matt')

# Ensure the output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

def fetch_summary_from_s3(file_number):
    url = f"https://edx-ai-class.s3.us-west-1.amazonaws.com/BLOG_SUMMARIES/blog_summary_{file_number}.txt"
    try:
        response = requests.get(url)
        response.raise_for_status()
        content = response.text.strip()
        if not content:
            logging.error(f"File is empty: {url}")
            return None
        return content
    except requests.exceptions.RequestException as e:
        logging.error(f"Failed to fetch file from {url}: {str(e)}")
        return None

def generate_blog_post(summary, retries=2):
    system_prompt = """You are a highly skilled content writer with expertise in creating engaging and informative blog posts. Your task is to expand upon provided summaries, transforming them into well-structured and captivating articles. Follow the detailed instructions and maintain a consistent, engaging tone throughout the content.

Instructions for generating a blog post:
1. **Introduction**: Craft a compelling introduction that introduces the topic and grabs the reader's attention. Set the context for the discussion and hint at what the reader will gain from the post.
2. **Body**: Elaborate on each key point from the summary. Use relevant examples, anecdotes, or data to enrich the content and provide valuable insights. Ensure each point transitions smoothly to the next.
3. **Tone and Style**: Maintain a consistent tone that resonates with a broad audience while providing detailed insights that enthusiasts will appreciate. Be informative yet engaging.
4. **Audience Engagement**: Incorporate questions, calls-to-action, or prompts for comments to engage the audience and encourage interaction. Make the reader feel involved.
5. **Conclusion**: Summarize the main points of the blog post and offer a final thought or call to action that leaves the reader thinking or motivated to act.
6. **SEO Considerations**: Integrate relevant keywords naturally throughout the post to enhance search engine visibility without compromising the quality and readability of the content.

Ensure the narrative flows logically and cohesively from the introduction to the conclusion, creating an engaging and informative reading experience.
"""

    user_prompt = f"""Using the provided summary below, generate a comprehensive blog post. Focus on expanding each point with detailed insights and context. Ensure the narrative flows logically from the introduction to the conclusion, creating an engaging and informative reading experience.

---BEGIN SUMMARY---
{summary}
---END SUMMARY---
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    for attempt in range(retries + 1):
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                max_tokens=1500,
                temperature=0.85
            )
            return response.choices[0].message.content.strip()
        except (APIConnectionError, RateLimitError, APIError) as e:
            logging.error(f"OpenAI API request failed (Attempt {attempt + 1}): {str(e)}")
            if attempt == retries:
                return None
        except Exception as e:
            logging.error(f"An unexpected error occurred (Attempt {attempt + 1}): {str(e)}")
            if attempt == retries:
                return None

def write_blog_post(content, file_number):
    output_filename = f"GPT_3.5_{file_number}.txt"
    output_path = os.path.join(output_directory, output_filename)
    try:
        with open(output_path, 'w', encoding='utf-8') as file:
            file.write(content)
            logging.info(f"Blog post written to {output_path}")
    except Exception as e:
        logging.error(f"Failed to write blog post to {output_path}: {str(e)}")

def main():
    start_time = time.time()
    file_count = 0

    for file_number in range(1, 101):  # Sequentially from 1 to 100
        logging.info(f"Processing file number: {file_number}")
        summary = fetch_summary_from_s3(file_number)
        if summary:
            blog_post = generate_blog_post(summary)
            if blog_post:
                write_blog_post(blog_post, file_number)
                file_count += 1

    elapsed_time = time.time() - start_time
    logging.info(f"Processed {file_count} files in {elapsed_time:.2f} seconds.")

if __name__ == '__main__':
    main()

2024-05-20 19:26:04,253 - INFO - Processing file number: 1
2024-05-20 19:26:15,739 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-20 19:26:15,754 - INFO - Blog post written to 3.5T Blog Matt/GPT_3.5_1.txt
2024-05-20 19:26:15,755 - INFO - Processing file number: 2
2024-05-20 19:26:26,075 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-20 19:26:26,078 - INFO - Blog post written to 3.5T Blog Matt/GPT_3.5_2.txt
2024-05-20 19:26:26,079 - INFO - Processing file number: 3
2024-05-20 19:26:34,926 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-20 19:26:34,929 - INFO - Blog post written to 3.5T Blog Matt/GPT_3.5_3.txt
2024-05-20 19:26:34,930 - INFO - Processing file number: 4
2024-05-20 19:26:44,268 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-20 19:26:44,271 - INFO - Blog post written to 3.5T Blog